# SCGNN Test - Trafik - PCA

In [1]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src import config
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader
import torch

In [2]:
df = pd.read_parquet(config.DATA_INTERIM / 'data.parquet')
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [3]:
PCA_COMPONENTS = 5
dataset = TrafficDataset(data_array, X_STEP, Y_STEP, pca_components=PCA_COMPONENTS)
sensors = dataset.sensors
edges = []
for i in range(sensors - 1):
    edges.append([i, i + 1])
    edges.append([i + 1, i])
edge_index = torch.tensor(edges, dtype=torch.long).t()
predictor = TrafficPredictor('scgnn', sensors, X_STEP, Y_STEP, edge_index=edge_index, pca_model=dataset.pca if True else None)
predictor.load_model(config.PROJECT_ROOT.joinpath('models/scgnn_trafik_pca.pt'))
loader = DataLoader(dataset, batch_size=10, num_workers=8)
preds = predictor.inference(loader)
display(pd.DataFrame(preds.squeeze()).head().iloc[:, :10])


,0,1,2,3,4
0,1543.317383,2000.947021,1699.084229,989.963867,305.120911
1,-0.229755,3723.209473,2973.778809,-0.229755,177.650681
2,826.532837,3117.689453,2668.276367,249.231125,413.374573
3,2286.502441,2375.401611,2219.376465,564.167908,250.440109
4,3574.255615,1658.279297,1676.647095,732.497742,132.557495
